# Finetuning AlexNet

Custom Dataset Class

In [2]:
import glob
import os
import cv2
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset


class CostumDataset(Dataset):

    def __init__(self, img_dir, img_size=100, augmentation=False):

        self.img_dir = img_dir
        self.img_size = img_size
        self.augmentation = augmentation

        img_data = []
        labels = []
        dirs = []
        for i, folder in enumerate(os.listdir(img_dir)):
            dirs.append(folder)
            for img in glob.glob(os.path.join(img_dir, folder, "*.jpg")):
                img_data.append(cv2.imread(img))
                labels.append(i)

        self.images = img_data
        self.labels = labels
        self.class_names = dirs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        transform_list = list()
        transform_list.append(T.ToTensor())
        transform_list.append(T.Resize(self.img_size))
        if self.augmentation == True:
            BILINEAR = T.InterpolationMode.BILINEAR
            aug_trans = [torchvision.transforms.RandomPerspective(interpolation=BILINEAR),
                  torchvision.transforms.RandomRotation(15, interpolation=BILINEAR),
                  torchvision.transforms.RandomHorizontalFlip(),
                  torchvision.transforms.RandomErasing()]
            transform_list.extend(aug_trans)

        data_transforms = T.Compose(transform_list)
        image = data_transforms(image)
        return image, label

    def get_class_names(self):
        return self.class_names


Finetuned AlexNet Model

In [3]:
def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  alexnet = torchvision.models.alexnet(pretrained=True)

  # get the number of neurons in the second last layer
  in_features = alexnet.classifier[6].in_features

  # re-initalize the output layer
  alexnet.classifier[6] = torch.nn.Linear(in_features=in_features,
                                          out_features=num_classes)

  return alexnet

In [4]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [5]:
def get_optimizer(model, lr, wd, momentum):
  # two groups of weights, one for the newly initialized layer and the other for rest of the layers of the network

  final_layer_weights = []
  rest_of_the_net_weights = []

  # iterate through the layers of the network
  for name, param in model.named_parameters():
    if name.startswith('classifier.6'):
      final_layer_weights.append(param)
    else:
      rest_of_the_net_weights.append(param)

  # distinct learning rates to each group of parameters
  optimizer = torch.optim.SGD([
      {'params': rest_of_the_net_weights},
      {'params': final_layer_weights, 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)

  return optimizer

Traning and Test Step

In [1]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
  samples = 0.0
  cumulative_loss = 0.0
  cumulative_accuracy = 0.0

  net.train()
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)

    outputs = net(inputs)
    loss = cost_function(outputs,targets)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    samples += inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(dim=1)

    # compute training accuracy
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

def test_step(net, data_loader, cost_function, device='cuda'):
  samples = 0.0
  cumulative_loss = 0.0
  cumulative_accuracy = 0.0

  net.eval()
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = net(inputs)
      loss = cost_function(outputs, targets)

      samples+=inputs.shape[0]
      cumulative_loss += loss.item()
      _, predicted = outputs.max(1)

      # compute accuracy
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

Main

In [2]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as TabError


# hyperparameters
batch_size = 4
learning_rate=0.001
weight_decay=0.000001
momentum=0.9
epochs=50
num_classes = 10

img_dir_train = 'data/scene/train'
img_dir_test = 'data/scene/test'

# device to use
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'


# instantiates datasets
full_training_dataset = CostumDataset(img_dir_train, 100, augmentation=True)
scene_dataset_test = CostumDataset(img_dir_test, 100)

# instantiates dataloaders
train_loader = torch.utils.data.DataLoader(full_training_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(scene_dataset_test, batch_size=batch_size, shuffle=False)

# instantiates the model
net = initialize_alexnet(num_classes=num_classes).to(device)

# instantiates the optimizer
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

# instantiates the cost function
cost_function = get_cost_function()

# perform a preliminar step
print('Before training:')
train_loss, train_accuracy = test_step(net, train_loader, cost_function, device=device)
# test_loss, test_accuracy = test_step(net, test_loader, cost_function, device=device)
print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
# print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')

epochs=50
for e in range(epochs):
  train_loss, train_accuracy = training_step(net, train_loader, optimizer, cost_function, device=device)
  # test_loss, test_accuracy = test_step(net, test_loader, cost_function, device=device)
  print('Epoch: {:d}'.format(e+1))
  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  # print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

# perform final test step and print the final metrics
print('After training:')
train_loss, train_accuracy = test_step(net, train_loader, cost_function, device=device)
test_loss, test_accuracy = test_step(net, test_loader, cost_function, device=device)
print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')


NameError: name 'CostumDataset' is not defined

Loading of the model and Evaluation

In [12]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
model = initialize_alexnet(num_classes=10).to(device)
model.load_state_dict(torch.load('state_dict_model.pt', map_location=torch.device(device)))

print('After training:')
train_loss, train_accuracy = test_step(model, train_loader, cost_function, device=device)
test_loss, test_accuracy = test_step(model, test_loader, cost_function, device=device)
print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')

c:\Users\nicol\OneDrive\Desktop\Assignment2\A2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nicol\OneDrive\Desktop\Assignment2\A2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


After training:


c:\Users\nicol\OneDrive\Desktop\Assignment2\A2\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


	Training loss 1.92563, Training accuracy 9.19
	Test loss 2.86072, Test accuracy 7.50
-----------------------------------------------------
